In [7]:
import scanpy as sc
import anndata as ad
import skimage
import matplotlib.pyplot as plt
import pandas as pd
import celltypist
from celltypist import models
import mygene

In [8]:
adata = sc.read_h5ad("Chrishna-dataset2.h5ad")
#print(adata.uns.keys())
mg = mygene.MyGeneInfo()
query = mg.querymany(adata.var_names.tolist(), scopes="ensembl.gene", fields="symbol", species="human")
mapping = {item['query']: item.get('symbol', item['query']) for item in query}
adata.var_names = [mapping.get(gene, gene) for gene in adata.var_names]


Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
querying 1-1000 ...
HTTP Request: POST https://mygene.info/v3/query/ "HTTP/1.1 200 OK"
querying 1001-2000 ...
HTTP Request: POST https://mygene.info/v3/query/ "HTTP/1.1 200 OK"
querying 2001-3000 ...
HTTP Request: POST https://mygene.info/v3/query/ "HTTP/1.1 200 OK"
querying 3001-4000 ...
HTTP Request: POST https://mygene.info/v3/query/ "HTTP/1.1 200 OK"
querying 4001-5000 ...
HTTP Request: POST https://mygene.info/v3/query/ "HTTP/1.1 200 OK"
querying 5001-6000 ...
HTTP Request: POST https://mygene.info/v3/query/ "HTTP/1.1 200 OK"
querying 6001-7000 ...
HTTP Request: POST https://mygene.info/v3/query/ "HTTP/1.1 200 OK"
querying 7001-8000 ...
HTTP Request: POST https://mygene.info/v3/query/ "HTTP/1.1 200 OK"
querying 8001-9000 ...
HTTP Request: POST https://mygene.info/v3/query/ "HTTP/1.1 200 OK"
querying 9001-10000 ...
HTTP Request: POS

In [9]:
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)


In [27]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.raw = adata.copy()
print(adata.raw)

adata.X seems to be already log-transformed.


Raw AnnData with n_obs × n_vars = 9706 × 4386
    var: 'feature_is_filtered', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std', 'feature_name', 'feature_reference', 'feature_biotype', 'feature_length', 'feature_type', 'n_cells'
    varm: 'PCs'


In [28]:
from scipy import sparse
import numpy as np

if sparse.issparse(adata.raw.X):
    raw_matrix = adata.raw.X.toarray()
else:
    raw_matrix = adata.raw.X

print("NaNs found:", np.isnan(raw_matrix).sum())
raw_matrix = np.nan_to_num(raw_matrix, nan=0.0)
adata.raw._X = raw_matrix

NaNs found: 15708


In [29]:
print(adata.shape)
print(adata.var.head())
sc.pp.filter_cells(adata, min_genes=200)
cells:sc.pp.filter_genes(adata,min_cells=3)
print(adata)

(9706, 4386)
       feature_is_filtered     means  dispersions  dispersions_norm  \
NOC2L                False  0.244551     2.136677          0.535014   
PERM1                False  0.018455     2.245080          0.802369   
HES4                 False  0.072697     2.265671          0.853154   
ISG15                False  0.070733     2.244208          0.800219   
AGRN                 False  0.040015     2.222369          0.746357   

               mean       std feature_name feature_reference feature_biotype  \
NOC2L -1.366024e-10  0.393818        NOC2L    NCBITaxon:9606            gene   
PERM1  1.271660e-11  0.103728        PERM1    NCBITaxon:9606            gene   
HES4   3.055579e-12  0.206338         HES4    NCBITaxon:9606            gene   
ISG15 -5.165277e-11  0.205172        ISG15    NCBITaxon:9606            gene   
AGRN  -3.644228e-11  0.153381         AGRN    NCBITaxon:9606            gene   

      feature_length    feature_type  n_cells  
NOC2L           1244  protein_c

In [14]:
models.download_models()
model = models.Model.load(model="Immune_All_Low.pkl")

📂 Storing models in C:\Users\workexperience25\.celltypist\data\models
⏩ Skipping [1/54]: Immune_All_Low.pkl (file exists)
⏩ Skipping [2/54]: Immune_All_High.pkl (file exists)
⏩ Skipping [3/54]: Adult_COVID19_PBMC.pkl (file exists)
⏩ Skipping [4/54]: Adult_CynomolgusMacaque_Hippocampus.pkl (file exists)
⏩ Skipping [5/54]: Adult_Human_MTG.pkl (file exists)
⏩ Skipping [6/54]: Adult_Human_PancreaticIslet.pkl (file exists)
⏩ Skipping [7/54]: Adult_Human_PrefrontalCortex.pkl (file exists)
⏩ Skipping [8/54]: Adult_Human_Skin.pkl (file exists)
⏩ Skipping [9/54]: Adult_Human_Vascular.pkl (file exists)
⏩ Skipping [10/54]: Adult_Mouse_Gut.pkl (file exists)
⏩ Skipping [11/54]: Adult_Mouse_OlfactoryBulb.pkl (file exists)
⏩ Skipping [12/54]: Adult_Pig_Hippocampus.pkl (file exists)
⏩ Skipping [13/54]: Adult_RhesusMacaque_Hippocampus.pkl (file exists)
⏩ Skipping [14/54]: Autopsy_COVID19_Lung.pkl (file exists)
⏩ Skipping [15/54]: COVID19_HumanChallenge_Blood.pkl (file exists)
⏩ Skipping [16/54]: COVID1

In [30]:

results = celltypist.annotate(adata, model=model)


🔬 Input data has 9706 cells and 4386 genes
🔗 Matching reference genes in the model
🧬 1787 features used for prediction
⚖️ Scaling input data
🖋️ Predicting labels


ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
adata.obs["cell_type"] = results.predicted_labels
print(results.predicted_labels)

In [ ]:
sc.pl.umap(adata, color="cell_type")